# 🛡️ Análise de Risco em Rede Militar com `shortestPaths` + Visualização

## 🧠 Pergunta:

Em uma rede de comunicações militares, foi detectada uma invasão no nó `"Base Comprometida"`.  
**Quais bases e instalações podem ser afetadas (direta ou indiretamente) pela comunicação dessa base, em até 2 conexões?**

Vamos modelar essa rede como um grafo direcionado e utilizar o algoritmo `shortestPaths` do GraphFrames.


In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder \
    .appName(("RedeMilitarShortestPath")) \
    .config("spark.jars.packages", "graphframes:graphframes:0.8.4-spark3.5-s_2.12") \
    .getOrCreate()

In [6]:
from graphframes import GraphFrame


In [7]:
bases = [
    "Comando Central", "Base Norte", "Base Sul", "Base Leste", "Base Oeste",
    "Base Avancada A", "Base Avancada B", "Radar Alfa", "Radar Bravo",
    "Hangar 1", "Hangar 2", "Arsenal", "Bunker", "Base Comprometida", 
    "Sala de Comunicacao", "Centro de Treinamento", "Controle Aereo", 
    "Posto de Fronteira", "Torre de Vigia", "Laboratorio"
]

In [10]:
vertices = spark.createDataFrame([(b,) for b in bases], ["id"])
vertices.show()

+--------------------+
|                  id|
+--------------------+
|     Comando Central|
|          Base Norte|
|            Base Sul|
|          Base Leste|
|          Base Oeste|
|     Base Avancada A|
|     Base Avancada B|
|          Radar Alfa|
|         Radar Bravo|
|            Hangar 1|
|            Hangar 2|
|             Arsenal|
|              Bunker|
|   Base Comprometida|
| Sala de Comunicacao|
|Centro de Treinam...|
|      Controle Aereo|
|  Posto de Fronteira|
|      Torre de Vigia|
|         Laboratorio|
+--------------------+



In [13]:
edges_list_dir = [
    ("Comando Central", "Base Norte"),
    # Como se estivesse invertendo ( "Base Norte","Comando Central"), 

    ("Comando Central", "Base Sul"),
    ("Base Norte", "Radar Alfa"),
    ("Base Sul", "Radar Bravo"),
    ("Base Norte", "Base Avancada A"),
    ("Base Sul", "Base Avancada B"),
    ("Base Avancada A", "Hangar 1"),
    ("Base Avancada B", "Hangar 2"),
    ("Radar Alfa", "Controle Aereo"),
    ("Radar Bravo", "Controle Aereo"),
    ("Controle Aereo", "Sala de Comunicacao"),
    ("Sala de Comunicacao", "Base Comprometida"),
    ("Base Comprometida", "Arsenal"),
    ("Base Comprometida", "Bunker"),
    ("Base Comprometida", "Posto de Fronteira"),
    ("Arsenal", "Laboratorio"),
    ("Bunker", "Torre de Vigia"),
    ("Base Comprometida", "Centro de Treinamento")
]

In [15]:
edges_list_nondir = edges_list_dir + [(dst, src) for (src, dst) in edges_list_dir]


In [17]:
edges = spark.createDataFrame(edges_list_nondir, ['src','dst'])

In [18]:
g = GraphFrame(vertices,edges)

In [19]:
shortest = g.shortestPaths(landmarks=['Base Comprometida'])

In [20]:
shortest.select("id", "distances").orderBy("id").show(truncate=False)

+---------------------+------------------------+
|id                   |distances               |
+---------------------+------------------------+
|Arsenal              |{Base Comprometida -> 1}|
|Base Avancada A      |{Base Comprometida -> 5}|
|Base Avancada B      |{Base Comprometida -> 5}|
|Base Comprometida    |{Base Comprometida -> 0}|
|Base Leste           |{}                      |
|Base Norte           |{Base Comprometida -> 4}|
|Base Oeste           |{}                      |
|Base Sul             |{Base Comprometida -> 4}|
|Bunker               |{Base Comprometida -> 1}|
|Centro de Treinamento|{Base Comprometida -> 1}|
|Comando Central      |{Base Comprometida -> 5}|
|Controle Aereo       |{Base Comprometida -> 2}|
|Hangar 1             |{Base Comprometida -> 6}|
|Hangar 2             |{Base Comprometida -> 6}|
|Laboratorio          |{Base Comprometida -> 2}|
|Posto de Fronteira   |{Base Comprometida -> 1}|
|Radar Alfa           |{Base Comprometida -> 3}|
|Radar Bravo        

In [21]:
from pyspark.sql.functions import col

In [22]:

shortest.filter (
    (col('distances')['Base Comprometida'] == 1) | 
    (col('distances')['Base Comprometida'] == 2)
).select("id", "distances").orderBy("id").show(truncate=False)

+---------------------+------------------------+
|id                   |distances               |
+---------------------+------------------------+
|Arsenal              |{Base Comprometida -> 1}|
|Bunker               |{Base Comprometida -> 1}|
|Centro de Treinamento|{Base Comprometida -> 1}|
|Controle Aereo       |{Base Comprometida -> 2}|
|Laboratorio          |{Base Comprometida -> 2}|
|Posto de Fronteira   |{Base Comprometida -> 1}|
|Sala de Comunicacao  |{Base Comprometida -> 1}|
|Torre de Vigia       |{Base Comprometida -> 2}|
+---------------------+------------------------+



In [23]:
## A MEsma coisa
shortest.filter (
    (col('distances')['Base Comprometida'] <= 3) 
).select("id", "distances").orderBy("id").show(truncate=False)

+---------------------+------------------------+
|id                   |distances               |
+---------------------+------------------------+
|Arsenal              |{Base Comprometida -> 1}|
|Base Comprometida    |{Base Comprometida -> 0}|
|Bunker               |{Base Comprometida -> 1}|
|Centro de Treinamento|{Base Comprometida -> 1}|
|Controle Aereo       |{Base Comprometida -> 2}|
|Laboratorio          |{Base Comprometida -> 2}|
|Posto de Fronteira   |{Base Comprometida -> 1}|
|Radar Alfa           |{Base Comprometida -> 3}|
|Radar Bravo          |{Base Comprometida -> 3}|
|Sala de Comunicacao  |{Base Comprometida -> 1}|
|Torre de Vigia       |{Base Comprometida -> 2}|
+---------------------+------------------------+



In [24]:
vertices = [
    "Base Comprometida", "Base Norte", "Radar Alfa", "Posto de Vigia",
    "Hangar Bravo", "Arsenal Secundario", "Estacao Delta", "Laboratorio Quimico",
    "Central de Comunicacoes", "Comando Regional",
    "Base Leste", "Base Sul", "Deposito Avancado", "Unidade Medica",
    "Torre Eco", "Hangar Alpha", "Oficina Tecnica"
]
v_df = spark.createDataFrame([(v,) for v in vertices], ["id"])

# 2. Arestas direcionadas originais
e_df = spark.createDataFrame([
    ("Central de Comunicacoes", "Base Comprometida"),
    ("Base Comprometida", "Base Norte"),
    ("Base Norte", "Radar Alfa"),
    ("Radar Alfa", "Posto de Vigia"),
    ("Base Norte", "Hangar Bravo"),
    ("Hangar Bravo", "Arsenal Secundario"),
    ("Base Norte", "Estacao Delta"),
    ("Estacao Delta", "Laboratorio Quimico"),
    ("Comando Regional", "Central de Comunicacoes"),
    ("Base Leste", "Base Sul"),
    ("Base Sul", "Deposito Avancado"),
    ("Deposito Avancado", "Unidade Medica"),
    ("Base Leste", "Torre Eco"),
    ("Torre Eco", "Hangar Alpha"),
    ("Hangar Alpha", "Base Sul"),
    ("Torre Eco", "Oficina Tecnica"),
    ("Unidade Medica", "Oficina Tecnica")
], ["src", "dst"])

# 3. Duplicar arestas no sentido inverso
e_invertido = e_df.select(col("dst").alias("src"), col("src").alias("dst"))
e_nondir = e_df.union(e_invertido).distinct()


In [26]:
g2 = GraphFrame(v_df, e_nondir)

In [33]:
shortest = g2.shortestPaths(landmarks=['Base Comprometida', 'Base Norte'])

In [36]:
shortest.filter (
    (col('distances')['Base Comprometida'] <= 2) &
    (col('distances')['Base Norte'] <= 2) 
).select("id", "distances").orderBy("id").show(truncate=False)

+-----------------------+-----------------------------------------+
|id                     |distances                                |
+-----------------------+-----------------------------------------+
|Base Comprometida      |{Base Comprometida -> 0, Base Norte -> 1}|
|Base Norte             |{Base Comprometida -> 1, Base Norte -> 0}|
|Central de Comunicacoes|{Base Comprometida -> 1, Base Norte -> 2}|
|Estacao Delta          |{Base Comprometida -> 2, Base Norte -> 1}|
|Hangar Bravo           |{Base Comprometida -> 2, Base Norte -> 1}|
|Radar Alfa             |{Base Comprometida -> 2, Base Norte -> 1}|
+-----------------------+-----------------------------------------+

